In [9]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [10]:
model_trainer:
   root_dir: artifacts/model_trainer
   data_path: artifacts/data_transformation/samsum-test
   model_ckpt: google/pegasus-cnn_dailymail 


SyntaxError: invalid syntax (1219980802.py, line 1)

In [1]:
from textSummarizer.entity import *
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, get_size, create_directories


In [2]:
class ConfigurationManager:
    def __init__(self, config_path = CONFIG_FILE_PATH, params_path = PARAMS_FILE_PATH):

        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)

        create_directories([self.config.artifacts_root])

    def get_modeltrainer_config(self)-> ModelTrainerConfig:
        config = self.config.model_trainer
        
        params = self.params.TrainingArguments # training parameters

        create_directories([config.root_dir])

        model_trainer_config  = ModelTrainerConfig(
             root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.evaluation_strategy,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
            
        )

        return model_trainer_config


In [5]:
import os
os.environ["USE_TF"] = "0"

In [6]:

from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


from datasets import load_dataset, load_from_disk
import torch

In [7]:
import os

In [9]:
class ModelTrainer:
    def __init__(self, config:ModelTrainerConfig):
        self.config = config


    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_pegasus)

        #load_data
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        # Split into train and validation manually
        dataset_samsum_pt = dataset_samsum_pt['train'].train_test_split(test_size=0.1) # 90% train, 10% validation


        # Training Arguments
        # train_args = TrainingArguments( 
        #     output_dir=self.config.root_dir,  # to save the output in the model trainer folder
        #     num_train_epochs= self.config.num_train_epochs,
        #     warmup_steps=self.config.warmup_steps,
        #     per_device_train_batch_size =self.config.per_device_train_batch_size,
        #     weight_decay = self.config.weight_decay,
        #     logging_steps= self.config.logging_steps,
        #     evaluation_strategy= self.config.evaluation_strategy,
        #     eval_steps= self.config.eval_steps,
        #     save_steps = self.config.eval_steps,
        #     gradient_accumulation_steps = self.config.gradient_accumulation_steps)
  

        
        train_args = TrainingArguments(
            output_dir=str(self.config.root_dir), num_train_epochs=1, warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            evaluation_strategy='steps', eval_steps=500, save_steps=1e6, #1000000
            gradient_accumulation_steps=16,  fp16=True,  # <--- this saves 50% memory
            dataloader_pin_memory=False  # <--- small trick to reduce GPU RAM
        )

        #To train 
        trainer = Trainer(model=model_pegasus,  
                          tokenizer=tokenizer, 
                          args=train_args,
                          data_collator=seq2seq_data_collator,
                train_dataset=dataset_samsum_pt['train'], 
                eval_dataset=dataset_samsum_pt['test'])

        trainer.train()

        #save model and tokenizer
        model_pegasus.save_pretrained(os.path.join(str(self.config.root_dir), 'pegasus_samsum_model'))
        tokenizer.save_pretrained(os.path.join(str(self.config.root_dir), 'tokenizer'))

In [41]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments, DataCollatorForSeq2Seq
import torch
import os
from datasets import load_from_disk
from dataclasses import dataclass


class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_pegasus)

        # Load dataset
        dataset_samsum_pt = load_from_disk(self.config.data_path)
        print("dataset_samsum_pt--", dataset_samsum_pt)

        # Split into train and validation manually
        dataset_samsum_pt = dataset_samsum_pt['train'].train_test_split(test_size=0.1)  # 90% train, 10% validation
        print("dataset_samsum_pt ['train']--", dataset_samsum_pt['train'])

        # Training Arguments
        train_args = TrainingArguments(
            output_dir=str(self.config.root_dir), 
            num_train_epochs=1, warmup_steps=50,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            evaluation_strategy='steps', eval_steps=50, save_steps=1e6, #1000000
            gradient_accumulation_steps=16,  fp16=True,  # <--- this saves 50% memory
            dataloader_pin_memory=False  # <--- small trick to reduce GPU RAM
        )
        
        # print("train_args --", train_args)

        # Initialize Trainer
        trainer = Trainer(
            model=model_pegasus,
            tokenizer=tokenizer,
            args=train_args,
            data_collator=seq2seq_data_collator,
            train_dataset=dataset_samsum_pt['train'],
            eval_dataset=dataset_samsum_pt['test']
        )

        print("trainer---", trainer)

        # Start training
        trainer.train()

        # Save model and tokenizer
        model_pegasus.save_pretrained(os.path.join(str(self.config.root_dir), 'pegasus_samsum_model'))
        tokenizer.save_pretrained(os.path.join(str(self.config.root_dir), 'tokenizer'))




In [29]:
cofig = ConfigurationManager()
model_trainer_config = cofig.get_modeltrainer_config()

model_trainer  = ModelTrainer(config=model_trainer_config)
model_trainer.train()

[2025-04-26 16:19:09,561: INFO: common: created directory at: artifacts]
[2025-04-26 16:19:09,563: INFO: common: created directory at: artifacts/model_trainer]


d:\Ananconda\envs\venv\lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


dataset_samsum_pt-- Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'label'],
    num_rows: 819
})


KeyError: "Column train not in the dataset. Current columns in the dataset: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'label']"

In [11]:
import os
os.environ["USE_TF"] = "0"